In [1]:
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
from IPython import display 
from astropy.coordinates import SkyCoord, Angle
from astropy import units as u
from astropy.io import ascii
import fitsio

In [2]:
np.seterr(divide = 'ignore', invalid = "ignore") 

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Import Clusters

In [3]:
clumper = Table.read("DESI_clusters.fits").to_pandas()

In [4]:
len(clumper)

378413

In [5]:
clumper.columns

Index(['index', 'RA_central', 'DEC_central', 'z_median_central',
       'z_average_no_wt', 'z_average_prob', 'z_average_mass_prob',
       'z_std_central', 'z_std_no_wt', 'z_std_prob', 'z_std_mass_prob',
       'RELEASE', 'BRICKID', 'OBJID', 'MASKBITS', 'gid', 'mass_central',
       'neighbor_mass', 'local_neighbor_mass', 'ultra_local_neighbor_mass',
       'correction_factor', 'neighbors', 'local_neighbors',
       'ultra_local_neighbors', 'flag_foreground'],
      dtype='object')

In [6]:
table_of_centers = pd.read_csv("dr9_sweeps.csv")

In [7]:
len(table_of_centers)

723

# Import Quasars

### Info about data structure

In [24]:
qso1_f = fits.open("https://data.desi.lbl.gov/public/ets/target/catalogs/dr9/1.1.1/targets/main/resolve/dark/targets-dark-hp-0.fits")

In [25]:
qso1 = qso1_f[1].data

In [26]:
qso1.columns

ColDefs(
    name = 'RELEASE'; format = 'I'
    name = 'BRICKID'; format = 'J'
    name = 'BRICKNAME'; format = '8A'
    name = 'BRICK_OBJID'; format = 'J'
    name = 'MORPHTYPE'; format = '4A'
    name = 'RA'; format = 'D'; unit = 'deg'
    name = 'RA_IVAR'; format = 'E'; unit = '1/deg^2'
    name = 'DEC'; format = 'D'; unit = 'deg'
    name = 'DEC_IVAR'; format = 'E'; unit = '1/deg^2'
    name = 'DCHISQ'; format = '5E'
    name = 'EBV'; format = 'E'; unit = 'mag'
    name = 'FLUX_G'; format = 'E'; unit = 'nanomaggy'
    name = 'FLUX_R'; format = 'E'; unit = 'nanomaggy'
    name = 'FLUX_Z'; format = 'E'; unit = 'nanomaggy'
    name = 'FLUX_IVAR_G'; format = 'E'; unit = '1/nanomaggy^2'
    name = 'FLUX_IVAR_R'; format = 'E'; unit = '1/nanomaggy^2'
    name = 'FLUX_IVAR_Z'; format = 'E'; unit = '1/nanomaggy^2'
    name = 'MW_TRANSMISSION_G'; format = 'E'
    name = 'MW_TRANSMISSION_R'; format = 'E'
    name = 'MW_TRANSMISSION_Z'; format = 'E'
    name = 'FRACFLUX_G'; format = 'E'
    na

In [28]:
from desitarget.targetmask import desi_mask as Mx

In [29]:
is_QSO = (qso1["DESI_TARGET"] & Mx["QSO"]) != 0

In [30]:
is_QSO

array([False,  True, False, ...,  True, False, False])

In [31]:
qso = qso1[is_QSO]

In [36]:
qsos = pd.DataFrame(np.array([qso.RA, qso.DEC, qso.MASKBITS]).T, columns = ["RA", "DEC", "MASKBITS"])

### Actual import

In [77]:
qso_list = pd.DataFrame(np.empty((0, 3)), columns = ["RA", "DEC", "MASKBITS"])

In [78]:
import requests
import re
from desitarget.targetmask import desi_mask as Mx
r = requests.get('https://data.desi.lbl.gov/public/ets/target/catalogs/dr9/1.1.1/targets/main/resolve/dark/')
files = re.findall(r'targets.*\.fits\"', r.text)

In [79]:
pbar = display.ProgressBar(len(files))
pbar.display()
for index, row in enumerate(files):
    url_string = 'https://data.desi.lbl.gov/public/ets/target/catalogs/dr9/1.1.1/targets/main/resolve/dark/' + row[:-1]
    row_imported = fits.open(url_string)[1].data
    is_QSO = (row_imported["DESI_TARGET"] & Mx["QSO"]) != 0
    row_data = row_imported[is_QSO]
    row_data_arr = np.array([row_data.RA, row_data.DEC, row_data.MASKBITS]).T
    qso_list = qso_list.append(pd.DataFrame(row_data_arr, columns = ["RA", "DEC", "MASKBITS"]),ignore_index=True)
    pbar.progress = index

[===========================                                 ] 209/460

OSError: [Errno 122] Disk quota exceeded

In [81]:
qso_list.memory_usage()

Index            128
RA          24191056
DEC         24191056
MASKBITS    24191056
dtype: int64

In [80]:
len(qso_list)

3023882

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


In [41]:
qsos = qsos[1:]

In [43]:
qsosdf = pd.DataFrame(qsos, columns = [["RA", "DEC"]])

In [45]:
len(clumper)

378413

In [47]:
len(qsosdf)

89144700

In [51]:
c = SkyCoord(ra=clumper.RA_central.to_numpy()*u.degree, dec=clumper.DEC_central.to_numpy()*u.degree)
catalog = SkyCoord(ra=qsosdf.RA.to_numpy().flatten()*u.degree, dec=qsosdf.DEC.to_numpy().flatten()*u.degree)

max_sep = 5 * u.arcmin
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
sep_constraint = d2d < max_sep
c_matches = c[sep_constraint]
catalog_matches = catalog[idx[sep_constraint]]

In [52]:
len(c_matches)

282697

In [27]:
c_matches

<SkyCoord (ICRS): (ra, dec) in deg
    [(229.7428581 ,  33.27818602), (222.81534768,  32.53095549),
     (327.61956912, -13.70416816), (323.97063956, -14.96544896),
     (324.58403798, -14.28503777), (321.23259419, -13.51356379),
     (321.45219546, -11.90927138), (325.11497578, -12.19459545),
     (329.70469756, -13.91067397), (327.53462194, -14.85423406),
     (323.89611423, -14.88845964), (327.28514286, -14.3864015 ),
     (327.54695322, -14.79193057), (328.35930306, -13.94767357),
     (227.71468301,  33.52678071), (228.46282536,  34.0532943 ),
     (227.39779407,  31.61604071), (125.36336088,  15.94035768),
     (121.30862768,  16.81311757), (125.05653818,  16.21591371),
     (126.90664527,  15.29811826), (121.70097124,  15.83396433),
     (122.94044432,  18.1464285 ), (314.34315601, -63.17228615),
     (354.91273367,  14.46457456), (351.13636109,  10.32190136)]>

In [54]:
qsosdf.iloc[idx[sep_constraint]][0:50]

,RA,DEC
70181530,160.618485,32.940802
70231662,167.329202,33.245922
70234292,160.092709,34.089681
70224972,166.530914,33.564645
70179385,160.724682,33.209881
70159877,164.034140,34.016934
70226524,167.707698,32.397815
70172405,164.662905,34.056457
70240811,168.002172,33.689459
70189969,169.246229,33.263585


In [57]:
clumper[sep_constraint][["RA_central", "DEC_central", "OBJID", "z_median_central"]]

,RA_central,DEC_central,OBJID,z_median_central
14672,229.742858,33.278186,2365,0.088023
14780,222.815348,32.530955,3653,0.088847
138229,327.619569,-13.704168,579,0.201515
138252,323.970640,-14.965449,3414,0.221304
138302,324.584038,-14.285038,2658,0.185983
138343,321.232594,-13.513564,131,0.176117
138480,321.452195,-11.909271,172,0.210693
138505,325.114976,-12.194595,3053,0.024110
138618,329.704698,-13.910674,1296,0.195729
138644,327.534622,-14.854234,1445,0.074821


In [78]:
values = clumper.flag_foreground.copy()

In [84]:
values[clumper[sep_constraint][clumper[sep_constraint].OBJID.isin(starsdf.iloc[idx[sep_constraint]].OBJID.to_numpy().flatten())].index] = 4

In [85]:
values[clumper[sep_constraint][clumper[sep_constraint].OBJID.isin(starsdf.iloc[idx[sep_constraint]].OBJID.to_numpy().flatten())].index]

138229    4
138252    4
138302    4
138343    4
138480    4
138505    4
138618    4
138644    4
138750    4
138904    4
138973    4
138977    4
141948    4
142633    4
142695    4
191892    4
191934    4
191965    4
191990    4
191992    4
192203    4
204731    4
248328    4
248374    4
Name: flag_foreground, dtype: int64

In [88]:
clumper[clumper.flag_foreground == 3]

,index,RA_central,DEC_central,z_median_central,z_average_no_wt,z_average_prob,z_average_mass_prob,z_std_central,z_std_no_wt,z_std_prob,...,neighbor_mass,local_neighbor_mass,ultra_local_neighbor_mass,correction_factor,neighbors,local_neighbors,ultra_local_neighbors,flag_foreground,flag_duplicate,edge_mask


In [89]:
clumper.flag_foreground = values

In [91]:
clumper[clumper.flag_foreground == 4]

,index,RA_central,DEC_central,z_median_central,z_average_no_wt,z_average_prob,z_average_mass_prob,z_std_central,z_std_no_wt,z_std_prob,...,neighbor_mass,local_neighbor_mass,ultra_local_neighbor_mass,correction_factor,neighbors,local_neighbors,ultra_local_neighbors,flag_foreground,flag_duplicate,edge_mask
138229,27367,327.619569,-13.704168,0.201515,0.291710,0.235547,0.237030,0.075796,0.128909,0.085394,...,12.860968,6.531785e+12,5.363057e+12,1.500161,51.933333,25.433333,1.500000,4,0,1
138252,27390,323.970640,-14.965449,0.221304,0.288048,0.252235,0.251947,0.021925,0.087441,0.053099,...,13.014226,5.206643e+12,2.291461e+12,1.533956,113.266667,40.500000,4.133333,4,0,1
138302,27440,324.584038,-14.285038,0.185983,0.269829,0.225377,0.225748,0.177255,0.094290,0.064153,...,12.768627,4.215708e+12,3.031131e+12,1.474138,63.033333,27.233333,4.133333,4,0,1
138343,27481,321.232594,-13.513564,0.176117,0.237189,0.202530,0.203016,0.243432,0.103959,0.070258,...,12.778683,3.721868e+12,2.310177e+12,1.457834,59.166667,19.333333,1.366667,4,0,0
138480,27618,321.452195,-11.909271,0.210693,0.278397,0.235877,0.236685,0.047546,0.117797,0.072520,...,12.731397,2.961628e+12,1.037631e+12,1.515744,55.400000,22.866667,1.633333,4,0,1
138505,27643,325.114976,-12.194595,0.024110,0.213862,0.131154,0.131845,0.045382,0.138294,0.091017,...,12.872547,2.303157e+12,7.022381e+11,1.227572,367.133333,90.200000,7.833333,4,0,1
138618,27756,329.704698,-13.910674,0.195729,0.288193,0.224285,0.224276,0.134488,0.116450,0.071465,...,12.705113,2.435269e+12,1.434135e+12,1.490414,67.833333,26.000000,1.766667,4,0,1
138644,27782,327.534622,-14.854234,0.074821,0.209521,0.130728,0.131033,0.061975,0.122040,0.079404,...,12.597574,2.060663e+12,7.206029e+11,1.300188,116.300000,43.866667,4.500000,4,0,0
138750,27888,323.896114,-14.888460,0.345622,0.320544,0.320050,0.320010,0.320534,0.101798,0.082764,...,12.864168,2.399271e+12,5.245800e+11,1.763662,89.533333,27.700000,3.700000,4,0,1
138904,28042,327.285143,-14.386402,0.668506,0.619622,0.619659,0.621709,0.232162,0.147202,0.101256,...,12.560776,2.509498e+12,6.754676e+11,2.525588,16.833333,7.566667,0.333333,4,0,1
